To Do:
Convert scores to score minus user average. 
Rewrite mapping function to create a dataframe with anime names. 


In [1]:
import pandas as pd

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from scipy.sparse import csr_matrix

In [4]:
csvtoload = 'animeedascores.csv'# this is a subset of scores used for EDA and to design the system. Replace with a full csv, or to an sql database for the full dataset. 

In [5]:
ratings = pd.read_csv(csvtoload) # This is a subset of reviews with all 0's dropped. 
ratings['userid']=ratings['index']
ratings['score']=ratings['animescore']

In [6]:
ratings1 = ratings[ratings['score']!=0][['score','userid','animeid']]

In [7]:
ratings1.head()

,score,userid,animeid
11,9,SeinfeldHentai,31637
12,8,SeinfeldHentai,6547
13,9,SeinfeldHentai,889
14,10,SeinfeldHentai,1
15,9,SeinfeldHentai,2025


In [8]:
useravg =ratings1.groupby('userid').mean() # this gives me a dataframe with each user's avg score. 

In [10]:
useravgdict = {} # This is an empty dictionary, which we will use to store a user's name as the key, and their avg score as the value

In [11]:
for user in useravg.index.values:
    score=useravg.loc[user,'score']
    useravgdict[user]=score

In [12]:
useravgdict # checking to make sure our dict looks good

{'---L-AND-AME-4EV': 9.0,
 '--Chiharu--': 7.7980769230769234,
 '--FearLess---': 10.0,
 '--joker--': 9.3333333333333339,
 '--kirikito--': 8.1595744680851059,
 '--x-Onigiri': 9.1999999999999993,
 '-2Konomo2-': 9.1999999999999993,
 '-6927-': 9.0,
 '-Aego-': 7.9622641509433958,
 '-Ainariel': 8.3584905660377355,
 '-AisakaTaiga-': 8.7422680412371125,
 '-Alleyne-': 9.0344827586206904,
 '-AmandaAi-': 10.0,
 '-AnimeArtist-': 9.0,
 '-AnimeHead': 8.9583333333333339,
 '-Annibal': 7.806451612903226,
 '-Atma': 8.0952380952380949,
 '-Blackspace-': 8.0,
 '-Caio-': 9.9073359073359075,
 '-Castiel': 7.0566572237960337,
 '-Chiharu-': 8.0444444444444443,
 '-Cloudz-': 7.2592592592592595,
 '-Cronos-': 8.3636363636363633,
 '-Crystale': 7.1321585903083697,
 '-DARKER-': 10.0,
 '-Dark--Shadow-': 9.5,
 '-DarkClaw-': 8.7439024390243905,
 '-DarkFlame-': 7.1489361702127656,
 '-DarkMaster-': 8.2687500000000007,
 '-DarkTems-': 9.5,
 '-Deagle-': 8.2068965517241388,
 '-DownFall': 8.3333333333333339,
 '-EXdeath-': 9.7111

In [13]:
ratings1['useravg']= ratings1.userid.map(useravgdict) # now we make a column for a user's avg score. 

In [14]:
ratings1.head() # checking to confirm our data

,score,userid,animeid,useravg
11,9,SeinfeldHentai,31637,9.111111
12,8,SeinfeldHentai,6547,9.111111
13,9,SeinfeldHentai,889,9.111111
14,10,SeinfeldHentai,1,9.111111
15,9,SeinfeldHentai,2025,9.111111


In [15]:
ratings1['score_scaled']=ratings1['score']-ratings1['useravg'] # making a column of how much more or less than avg a user liked an anime

In [16]:
ratings1.head()# checking our data

,score,userid,animeid,useravg,score_scaled
11,9,SeinfeldHentai,31637,9.111111,-0.111111
12,8,SeinfeldHentai,6547,9.111111,-1.111111
13,9,SeinfeldHentai,889,9.111111,-0.111111
14,10,SeinfeldHentai,1,9.111111,0.888889
15,9,SeinfeldHentai,2025,9.111111,-0.111111


In [ ]:
#anime_pivot_table = ratings1.pivot_table(index='userid', columns = 'animeid',values='score_scaled')
# a pivot table  pandas function would be easier than what I do here but will run most systems out of memory. 

In [17]:
users = ratings.userid.unique()
animes = ratings.animeid.unique()

In [18]:
userid2idx = {o:i for i,o in enumerate(users)} # turn the user strings into a numerical value. 
animeid2idx = {o:i for i,o in enumerate(animes)} # remove missing anime numbers and re-order 

In [19]:
ratings1['anime_id'] = ratings1.animeid.apply(lambda x: animeid2idx[x]) #the above code set up mapping, this applies it. 
ratings1.userid = ratings1.userid.apply(lambda x: userid2idx[x])# the above code set up mapping, this applies it. 


We are using these mapping to generate which row and column each username and anime_id will go into. 

In [20]:


user_u = list(sorted(ratings1.animeid.unique()))
anime_id_u = list(sorted(ratings1.userid.unique()))

data = ratings1['score_scaled'].tolist() # you can change this back to ['score'] to test and see what the unscaled similarities are
row = ratings1.userid.astype('category')
col = ratings1.anime_id.astype('category')
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(users), len(animes)))

In [21]:
reviewmatrix =sparse_matrix.T 

In [22]:
#col

11           11
12           12
13           13
14           14
15           15
16           16
17           17
18           18
19           19
21           21
23           23
24           24
25           25
26           26
27           27
28           28
29           29
30           30
31           31
32           32
33           33
34           17
35           34
36           35
37           36
38           37
39           38
40           39
219         212
220         213
           ... 
8989016     606
8989017    2703
8989018     772
8989019     318
8989020      79
8989021    2102
8989022    1776
8989023    2052
8989024    1692
8989025     242
8989026     467
8989027     111
8989028       3
8989029      31
8989030     120
8989031     321
8989032    1589
8989033    1590
8989034      32
8989035     301
8989036     302
8989037     165
8989038     180
8989039     181
8989040    2110
8989041     202
8989042    2116
8989043    2505
8989052    1771
8989056     800
Name: anime_id, Length: 

In [23]:
#row

11             1
12             1
13             1
14             1
15             1
16             1
17             1
18             1
19             1
21             2
23             2
24             2
25             2
26             2
27             2
28             2
29             2
30             2
31             2
32             2
33             2
34             2
35             2
36             2
37             2
38             2
39             2
40             2
219            4
220            4
           ...  
8989016    90515
8989017    90515
8989018    90515
8989019    90515
8989020    90515
8989021    90515
8989022    90515
8989023    90515
8989024    90515
8989025    90515
8989026    90515
8989027    90515
8989028    90515
8989029    90515
8989030    90515
8989031    90515
8989032    90515
8989033    90515
8989034    90515
8989035    90515
8989036    90515
8989037    90515
8989038    90515
8989039    90515
8989040    90515
8989041    90515
8989042    90515
8989043    905

In [24]:
score_cos =cosine_similarity(reviewmatrix) # now we generatie our cosine of similarity

In [ ]:
#animeinfo= pd.read_csv('animeinfo3.csv')
#animenames=pd.merge(ratings1,animeinfo,left_on='anime_id',right_on='animeid') # 

In [ ]:
#ratings1.sort_values('anime_id')

In [25]:
inv_map = {v: k for k, v in  animeid2idx.items()} # we have to make an inverse map of the which row/column number is what anime number.


In [26]:
anime_names = pd.read_csv('animeinfo3.csv').set_index('animeid')['name'].to_dict() # Then we map out anime_id to anime name

In [27]:
anime_names[32878] # confirming that our mapping is correct by going to https://myanimelist.net/anime/32878

'ēlDLIVE'

In [28]:
anime_in_order = []

In [29]:
for  anime in list(inv_map.values()): # we are going through the inverse map, and making a list of the anime names in the row/column order. 
    try:
        anime_in_order.append(anime_names[anime])
    except:
        anime_in_order.append('unknownanime'+str(anime)) 

In [30]:
anime_in_order

['ēlDLIVE',
 'Boku no Hero Academia',
 'Kono Naka ni Hitori, Imouto ga Iru!',
 'Love Hina',
 'Midori no Hibi',
 'Needless',
 'Princess Lover!',
 'Zankyou no Terror',
 'Akikan!',
 'Seto no Hanayome',
 'Shuffle!',
 'Gate: Jieitai Kanochi nite, Kaku Tatakaeri 2nd Season',
 'Angel Beats!',
 'Black Lagoon',
 'Cowboy Bebop',
 'Darker than Black: Kuro no Keiyakusha',
 'Gate: Jieitai Kanochi nite, Kaku Tatakaeri',
 'One Punch Man',
 'Overlord',
 'Trigun',
 'Shouwa Genroku Rakugo Shinjuu',
 'Baccano!',
 'Code Geass: Hangyaku no Lelouch R2',
 'Elfen Lied',
 'Evangelion: 2.0 You Can (Not) Advance',
 'Fate/Zero',
 'Fate/Zero 2nd Season',
 'Ghost in the Shell',
 'Hellsing Ultimate',
 'Hotarubi no Mori e',
 'Hunter x Hunter (2011)',
 'Magi: The Kingdom of Magic',
 'Mononoke Hime',
 'Nichijou',
 'Rainbow: Nisha Rokubou no Shichinin',
 'Shingeki no Kyojin',
 'Tengen Toppa Gurren Lagann',
 'Top wo Nerae! & Top wo Nerae 2! Gattai Movie!!',
 'Usagi Drop',
 'Yojouhan Shinwa Taikei',
 'Black Bullet',
 'Bok

In [31]:
rev1_sim_cos_df = pd.DataFrame(score_cos, index=anime_in_order , columns=anime_in_order)

In [32]:
rev1_sim_cos_df.loc[:,'Girls und Panzer'].sort_values()[-20:]

Non Non Biyori: Okinawa e Ikukoto ni Natta                   0.064432
Wake Up, Girls! Seishun no Kage                              0.064726
Hidamari Sketch x ☆☆☆ Specials                               0.065780
Non Non Biyori Repeat OVA                                    0.066169
Non Non Biyori Repeat                                        0.068354
Non Non Biyori                                               0.069801
Yuru Yuri Nachuyachumi!                                      0.070839
Hidamari Sketch x 365                                        0.072859
Gochuumon wa Usagi desu ka??                                 0.073486
Yuru Yuri♪♪                                                  0.079418
Saki Achiga-hen: Episode of Side-A                           0.079778
Girls und Panzer der Film Specials                           0.080207
Hidamari Sketch x ☆☆☆                                        0.083120
Girls und Panzer: Fushou - Akiyama Yukari no Sensha Kouza    0.085316
Hidamari Sketch x Ho

In [33]:
rev1_sim_cos_df.loc[:,'Death Note'].sort_values()[-20:]

Mononoke Hime                         0.153273
Shingeki no Kyojin Season 2           0.156281
Great Teacher Onizuka                 0.156678
Tengen Toppa Gurren Lagann            0.160526
Boku dake ga Inai Machi               0.161512
Hellsing Ultimate                     0.162934
Fate/Zero                             0.175154
Clannad: After Story                  0.175703
Psycho-Pass                           0.178386
Sen to Chihiro no Kamikakushi         0.196155
Fate/Zero 2nd Season                  0.202692
One Punch Man                         0.205312
Hunter x Hunter (2011)                0.213298
Kiseijuu: Sei no Kakuritsu            0.222627
Steins;Gate                           0.264896
Shingeki no Kyojin                    0.265831
Fullmetal Alchemist: Brotherhood      0.309242
Code Geass: Hangyaku no Lelouch       0.327511
Code Geass: Hangyaku no Lelouch R2    0.329206
Death Note                            1.000000
Name: Death Note, dtype: float64

In [34]:
rev1_sim_cos_df.loc[:,'Bakemonogatari'].sort_values()[-20:]

Kizumonogatari III: Reiketsu-hen    0.197705
Ookami to Koushinryou               0.200260
Ookami to Koushinryou II            0.201316
Kara no Kyoukai 5: Mujun Rasen      0.204040
Fate/Zero                           0.207812
Tengen Toppa Gurren Lagann          0.210683
Mahou Shoujo Madoka★Magica          0.218667
Fate/Zero 2nd Season                0.229016
Katanagatari                        0.244833
Suzumiya Haruhi no Shoushitsu       0.250727
Kizumonogatari II: Nekketsu-hen     0.261660
Steins;Gate                         0.272186
Kizumonogatari I: Tekketsu-hen      0.276123
Hanamonogatari                      0.349847
Tsukimonogatari                     0.361698
Owarimonogatari                     0.363377
Nekomonogatari: Kuro                0.415256
Nisemonogatari                      0.516315
Monogatari Series: Second Season    0.534567
Bakemonogatari                      1.000000
Name: Bakemonogatari, dtype: float64

In [35]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]

Bleach                                                      0.089613
Ao no Exorcist                                              0.091284
Aldnoah.Zero 2nd Season                                     0.092552
Ore no Imouto ga Konnani Kawaii Wake ga Nai.                0.093213
Naruto: Shippuuden                                          0.093515
Sword Art Online: Extra Edition                             0.095120
Fairy Tail (2014)                                           0.101367
Black Bullet                                                0.103876
Mahouka Koukou no Rettousei                                 0.110608
Tokyo Ghoul                                                 0.115369
Charlotte                                                   0.116895
Highschool of the Dead                                      0.117798
Guilty Crown                                                0.131108
Tokyo Ghoul √A                                              0.135092
Mirai Nikki (TV)                  

In [ ]:
rev1_sim_cos_df.loc[:,'Sword Art Online'].sort_values()[-20:]